# Creating Tokens

In [222]:
with open("the-verdict(1).txt","r",encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of characters:", len(raw_text))
print(raw_text[:99])

Total number of characters: 20480
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


#### Goal is to tokenize all the characters in the txt file that can be later turned into embeddings for LLMs

In [224]:
import re

text = "Hello, world. This, is a test."
result = re.split(r'(\s)',text)

print(result) # result of individual words , whitespaces an punctuation characters

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [225]:
result = re.split(r'([,.]|\s)',text)
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [226]:
# This will prevent the spaces to be considered as tokens
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


In [227]:
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()|\-\s]+)', text)
result = [item.strip() for item in result if item.strip()]
print(result)


['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [228]:
preprocessed = re.split(r'([,.:;?_!"()|\-\s])',raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]

print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '-', '-', 'though', 'a', 'good', 'fellow', 'enough', '-', '-', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that']


In [229]:
print(len(preprocessed))

4692


 # Creating Token IDs

In [231]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

1163


In [232]:
 vocab = {token:integer for integer, token in enumerate(all_words)}

In [233]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break 

('!', 0)
('"', 1)
("'", 2)
("'Are", 3)
("'It's", 4)
("'coming'", 5)
("'done'", 6)
("'subject", 7)
("'technique'", 8)
("'way", 9)
('(', 10)
(')', 11)
(',', 12)
('-', 13)
('.', 14)
(':', 15)
(';', 16)
('?', 17)
('A', 18)
('Ah', 19)
('Among', 20)
('And', 21)
('Arrt', 22)
('As', 23)
('At', 24)
('Be', 25)
('Begin', 26)
('Burlington', 27)
('But', 28)
('By', 29)
('Carlo', 30)
('Chicago', 31)
('Claude', 32)
('Come', 33)
('Croft', 34)
('Destroyed', 35)
('Devonshire', 36)
("Don't", 37)
('Dubarry', 38)
('Emperors', 39)
('Florence', 40)
('For', 41)
('Gallery', 42)
('Gideon', 43)
('Gisburn', 44)
("Gisburn's", 45)
('Gisburns', 46)
('Grafton', 47)
('Greek', 48)
('Grindle', 49)
("Grindle's", 50)


In [234]:
class SimpleTokenizer:
    def __init__(self,vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
        
    def encode(self,text):
        preprocessed = re.split(r'([,.:;?_!"()\-\s])',text)
        processed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed if s.strip() and s in self.str_to_int]
        return ids
        
    def decode(self,ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r's+([,.?!"()\'])',r'\1', text)
        return text

#### Instantiate a tokenizer object from the tokenizer class

In [236]:
tokenizer = SimpleTokenizer(vocab)

text = """"" It's the last he painted, you know,"
             Mrs. Gisburn said to pardonable pride."""

ids = tokenizer.encode(text)
print(ids)

[1, 1, 69, 1016, 627, 552, 772, 12, 1157, 621, 12, 1, 81, 14, 44, 879, 1046, 781, 822, 14]


In [237]:
tokenizer.decode(ids)

'" " It\'s the last he painted , you know , " Mrs . Gisburn said to pardonable pride .'

# Special Context Tokens

### These are something that are used to deal with the unknown words which
### are not present in the training vocubulary or training data.

#### Here simpleTokenizer is modified

In [240]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>","<|unk|>"])

vocab = {token:integer for integer, token in enumerate(all_tokens)}

In [241]:
len(vocab.items())

1165

In [242]:
for i, item in enumerate(list(vocab.items())[-3:]):
    print(item)

('yourself', 1162)
('<|endoftext|>', 1163)
('<|unk|>', 1164)


In [243]:
# Modification in SimpleTokenizer

class SimpleTokenizerV2:
    def __init__(self,vocab):
        self.str_to_int = vocab
        self.int_to_str = {i: s for s, i in vocab.items()}
        
    def encode(self,text):
        preprocessed = re.split(r'([,.:;?_!"()\-\s])',text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed 
        ]
        ids = [self.str_to_int[s] for s in preprocessed if s.strip() and s in self.str_to_int]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

In [244]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "in the sunlit terrace of the palace."

text = " <|endoftext|> ".join((text1,text2))

print(text)


Hello, do you like tea? <|endoftext|> in the sunlit terrace of the palace.


In [245]:
tokenizer.encode(text)

[1164,
 12,
 378,
 1157,
 653,
 1004,
 17,
 1163,
 592,
 1016,
 986,
 1011,
 748,
 1016,
 1164,
 14]

In [246]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> in the sunlit terrace of the <|unk|>.'

# Byte Pair Encoding

In [248]:
!pip install tiktoken

In [249]:
import importlib
import tiktoken

In [250]:
tokenizer = tiktoken.get_encoding("gpt2")

In [251]:
text = (
    "Hello do you like tea? <|endoftext|> In the sunlit terraces"
    "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [252]:
strings = tokenizer.decode(integers)
print(strings)

Hello do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


# Creating Input-Target Pairs

In [254]:
with open("the-verdict(1).txt","r",encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5146


In [255]:
# Remove first 50 tokens from the dataset for demonstration purpose
enc_sample = enc_text[50:]

In [256]:
context_size = 4  #length of input

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:    {y}")

x: [290, 4920, 2241, 287]
y:    [4920, 2241, 287, 257]


In [257]:
for i in range(1, context_size+1):
    context = enc_sample[:i] # Input the llm will recieve
    desired = enc_sample[i] # Target
    
    print(context,"----->",desired)

[290] -----> 4920
[290, 4920] -----> 2241
[290, 4920, 2241] -----> 287
[290, 4920, 2241, 287] -----> 257


In [258]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


# Implementing Dataset

#### 1.Tokenize the entire text
#### 2.Use the Sliding Window to chunk into overlapping sequences of max_length
#### 3.Return the total number of rows in the dataset
#### 4.Return a single row from the dataset

In [261]:
!pip install torch

In [262]:
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1: i + max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
            
    def __len__(self):
        return len(self.input_ids)
        
    def __getitem__(self,idx):
        return self.input_ids[idx], self.target_ids[idx]
            
        

cessing

In [264]:
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                          stride=128, shuffle=True, drop_last=True,
                          num_workers=0):
    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers = num_workers

    )

    
    return dataloader

In [265]:
with open("the-verdict(1).txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

#### Convert Dataloader into a Python iterator to fetch the next entry via Python's build-in next() function

In [267]:
import torch
print("PyTorch version:", torch.__version__)
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

PyTorch version: 2.4.1+cpu
[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [268]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


#### The stride setting dictates the number of positions of inputs shift across batches, emulating a sliding window approach

In [270]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n",inputs)
print("\nTargets:\n",targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


# Token Embeddings

In [272]:
input_ids = torch.tensor([2,3,5,1])

In [273]:
vocab_size = 6 # Only 6 tokens are used instead of 50257 words
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [274]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [275]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [276]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


# Positional Embeddings

#### We now consider more realistic and useful embedding sizes and encode into 256-dimensional vector representation

In [279]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [280]:
# Initiatiate the data-loader

max_length = 4
dataloadder = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
input, targets = next(data_iter)

In [281]:
print(input)

tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])


In [284]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [288]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

In [298]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings)
print(pos_embeddings.shape)

tensor([[ 1.7375, -0.5620, -0.6303,  ..., -0.2277,  1.5748,  1.0345],
        [ 1.6423, -0.7201,  0.2062,  ...,  0.4118,  0.1498, -0.4628],
        [-0.4651, -0.7757,  0.5806,  ...,  1.4335, -0.4963,  0.8579],
        [-0.6754, -0.4628,  1.4323,  ...,  0.8139, -0.7088,  0.4827]],
       grad_fn=<EmbeddingBackward0>)
torch.Size([4, 256])
